In [1]:
import tensorflow as tf
import numpy as np
import vgg19
import matplotlib.pyplot as plt
import utils

In [2]:
i_max = 100

device = '/cpu:0'
show = False

In [3]:
def gram_matrix(F, N, M):
    Ft = tf.reshape(F, (M, N))
    if M < N:
        return tf.matmul(Ft,tf.transpose(Ft))
    else:
        return tf.matmul(tf.transpose(Ft), Ft)

In [4]:
def create_noise_image(width, height):
    image = np.random.uniform(-32,32, (1, height, width, 3)).astype('float32')
    
    if(show == True):
        plt.imshow(image.reshape(height,width,3))
        plt.show()
    return image

In [5]:
def get_layers(image, scope):      
    #vgg
    vgg = vgg19.Vgg19()
    
    with tf.name_scope(scope):
        vgg.build(image)

    #layers = [vgg.pool4, vgg.pool3, vgg.pool2, vgg.pool1, vgg.conv1_1]
    layers = [vgg.conv1_1]
    
    return layers

In [6]:
with tf.device(device):
    sess = tf.InteractiveSession()
    
    
    #create target image
    target_img = (create_noise_image(224, 224))
    
    #create variables                 
    target = tf.Variable(target_img, tf.float32)
    
    #target vgg
    target_layers = get_layers(target, "target")
    
    sess.run(tf.global_variables_initializer())
    
    gram = []
    #go through layers
    for tl in target_layers:
        #get dimensions
        dimensions = tl.get_shape().as_list()
        size = dimensions[3]*dimensions[2]*dimensions[1]
        
        #get gram
        g = sess.run(gram_matrix(tl, dimensions[3], dimensions[2]*dimensions[1]))
        gram.append(g)
        
        rows = g.shape[0]
        cols = g.shape[1]
        
        if(show == True):
            for i in range(rows):
                for j in range(cols):
                    print(g[i][j], end = '')
                    print(' ', end = '')
                print(';', end = '')
        
        print(sess.run(tf.square(g)))
        print(sess.run(tf.reduce_sum(tf.square(g))))
        

/home/rwjoverwater/in4015/vgg19.npy
npy file loaded
build model started
build model finished: 0s
[[  1.56493328e+24   2.41386342e+22   5.71087958e+22 ...,   3.23511508e+22
    1.65885553e+23   4.25030890e+23]
 [  2.41386342e+22   1.47041661e+22   7.64544246e+21 ...,   1.54983880e+21
    1.99992790e+22   2.22578477e+22]
 [  5.71087958e+22   7.64544246e+21   2.01847530e+22 ...,   3.31564265e+21
    5.95728728e+22   8.45624689e+22]
 ..., 
 [  3.23511508e+22   1.54983880e+21   3.31564265e+21 ...,   3.61423508e+22
    2.72415108e+22   3.12417746e+22]
 [  1.65885553e+23   1.99992790e+22   5.95728728e+22 ...,   2.72415108e+22
    6.27227345e+23   6.07713356e+23]
 [  4.25030890e+23   2.22578477e+22   8.45624689e+22 ...,   3.12417746e+22
    6.07713356e+23   8.57581422e+23]]
2.41307e+26
